In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as la
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
import time

In [2]:
def tsp(M):
    return np.transpose(M)

def threshold(v,θ):
    n = len(v)
    H_v = np.zeros(n)
    for i in range(n):
        if v[i] < θ:
            H_v[i] = 0
        else:
            H_v[i] = v[i]
    Z = np.sum(H_v)
    for i in range(n):
        H_v[i] = H_v[i]/Z
    return H_v

def mvpo_egd(μ,Σ,γ,K,η):
    N = len(μ)
    w_hist = np.zeros((K,N))
    u_hist = np.zeros(K)
    w_curr = np.ones(N)/N
    for k in range(K):
        g_curr = γ*Σ@w_curr-μ
        w_next = np.zeros(N)
        w_norm = 0
        for i in range(N):
            w_norm += w_curr[i]*np.exp(-η*g_curr[i])
        for i in range(N):
            w_next[i] = (w_curr[i]*np.exp(-η*g_curr[i]))/w_norm
        w_hist[k,:] = w_next
        u_hist[k] = w_next@μ-0.5*γ*w_next@Σ@w_next
        w_curr = w_next
    return w_hist[-1], w_hist, u_hist

In [3]:
r = np.load(r"C:\Users\zhubr\OneDrive\Desktop\ECON 492\data\old-i\r.npz")['r']
F = np.load(r"C:\Users\zhubr\OneDrive\Desktop\ECON 492\data\old-i\F.npz")['F'] 
T = r.shape[0]
N = r.shape[1]

In [4]:
H = 7
T_train = 300
T_test = [T_train]
H_curr = T_train
while H_curr <= (T-2*H):
    H_curr += H
    T_test.append(H_curr)
T_test = np.array(T_test)
N_test = len(T_test)
print(T_test)
print(N_test)

[300 307 314 321 328 335 342 349 356 363 370 377 384 391 398 405]
16


In [5]:
np.random.seed(492)

μ_f = np.mean(F,axis=0)
Σ_f = np.cov(tsp(F))

J = 100

# θ = np.random.normal(loc=0,scale=0.5,size=J)
# η = np.random.normal(loc=0,scale=0.5,size=J)

θ = np.random.uniform(-1,1,size=J)
η = np.random.uniform(-1,1,size=J)

# γ = np.random.exponential(scale=0.25,size=J)

γ = np.random.uniform(0,1,size=J)

In [6]:
N = 100
N_IU = 100
### CREATE INSTRUMENT ###

size = np.load(r"C:\Users\zhubr\OneDrive\Desktop\ECON 492\data\old-i\size.npz")['data']
supply = np.load(r"C:\Users\zhubr\OneDrive\Desktop\ECON 492\data\old-i\supply.npz")['data']
powind = np.load(r"C:\Users\zhubr\OneDrive\Desktop\ECON 492\data\old-i\powind.npz")['data']
momentum = np.load(r"C:\Users\zhubr\OneDrive\Desktop\ECON 492\data\old-i\momentum.npz")['data']

log_mktcap = np.zeros((N_test,N_IU))
log_supply = np.zeros((N_test,N_IU))
proof_of_work = np.zeros((N_test,N_IU))
beta_market = np.zeros((N_test,N_IU))
momentum_1w = np.zeros((N_test,N_IU))
for n in range(N_test):
    fm = np.load(r"C:\Users\zhubr\OneDrive\Desktop\ECON 492\data\old-i\test\fm-"+str(n)+".npz")
    for i in range(N_IU):
        log_mktcap[n,i] = np.log(size[T_test[n],i])
        log_supply[n,i] = np.log(supply[T_test[n],i])
        proof_of_work[n,i] = powind[i]
        beta_market[n,i] = fm['beta'][i,0]
        momentum_1w[n,i] = momentum[T_test[n],i]

log_mktcap_f = log_mktcap.flatten()
log_supply_f = log_supply.flatten()
proof_of_work_f = proof_of_work.flatten()
beta_market_f = beta_market.flatten()
momentum_1w_f = momentum_1w.flatten()

y = log_mktcap_f
X = np.column_stack((log_supply_f,proof_of_work_f,beta_market_f,momentum_1w_f))
mod = sm.OLS(y,X)
res = mod.fit()
log_mktcap_inst_f = mod.predict(res.params,X)
log_mktcap_inst = np.reshape(log_mktcap_inst_f,(N_test,N_IU))

In [7]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.981
Model:                            OLS   Adj. R-squared (uncentered):              0.981
Method:                 Least Squares   F-statistic:                          2.110e+04
Date:                Wed, 06 Apr 2022   Prob (F-statistic):                        0.00
Time:                        01:39:12   Log-Likelihood:                         -4009.4
No. Observations:                1600   AIC:                                      8027.
Df Residuals:                    1596   BIC:                                      8048.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.6907      0.017     40.319      0.000       0.657       0.724
x2             1.0091      0.208      4.856      0.000       0.602       1.417
x3             7.0477      0.323     21.822      0.000       6.414       7.681
x4             0.0601      0.026      2.320      0.020       0.009       0.111
==============================================================================
Omnibus:                       13.291   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               17.204
Skew:                           0.108   Prob(JB):                     0.000184
Kurtosis:                       3.460   Cond. No.                         88.3
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
y = []
X = []
Z = []

for n in range(N_test):
    
    r_test = r[T_test[n]-T_train:T_test[n],:]
    F_test = F[T_test[n]-T_train:T_test[n],:]
    
    μ_f = np.mean(F_test,axis=0)
    Σ_f = np.cov(tsp(F_test))

    rsfm = np.load(r"C:\Users\zhubr\OneDrive\Desktop\ECON 492\data\old-i\test\rsfm-"+str(n)+".npz")

    P = rsfm['prediction'][-1]
    avg_risk = np.mean(rsfm['sigma'],axis=1)

    for j in range(J):

        ### TILT DISTRIBUTION ###

        S = len(P)
        P_sort = np.argsort(P)
        avg_risk_sort = np.argsort(avg_risk)
        M_θ = 0
        for s in range(S):
            P_rank = np.where(P_sort==s)[0][0]+1
            M_θ += P[s]*np.exp(θ[j]*P_rank)
        P_θ = np.zeros(S)
        for s in range(S):
            P_rank = np.where(P_sort==s)[0][0]+1
            P_θ[s] = (P[s]*np.exp(θ[j]*P_rank))/M_θ
        M_θη = 0
        for s in range(S):
            avg_risk_rank = np.where(avg_risk_sort==s)[0][0]+1
            M_θη += P_θ[s]*np.exp(η[j]*avg_risk_rank)
        P_θη = np.zeros(S)
        for s in range(S):
            avg_risk_rank = np.where(avg_risk_sort==s)[0][0]+1
            P_θη[s] = (P_θ[s]*np.exp(η[j]*avg_risk_rank))/M_θη    
        P_tilt = P_θη

        ### COMPUTE OPTIMAL WEIGHT ###

        μ = np.zeros(N)
        Σ = np.zeros((N,N))
        for s in range(S):
            μ += P_tilt[s]*(rsfm['alpha'][s]+rsfm['beta'][s]@μ_f)
            Σ += P_tilt[s]*(np.outer(rsfm['alpha'][s],rsfm['alpha'][s])+np.outer(rsfm['alpha'][s],rsfm['beta'][s]@μ_f)
                             +np.outer(rsfm['beta'][s]@μ_f,rsfm['alpha'][s])+rsfm['beta'][s]@(Σ_f+np.outer(μ_f,μ_f))@tsp(rsfm['beta'][s])
                             +np.diag(rsfm['sigma'][s]**2))
        Σ= Σ-np.outer(μ,μ)
        μ = 1+μ/100
        Σ = Σ/10000
        μ = μ[:N_IU]
        Σ = Σ[:N_IU,:N_IU]

        K = 500
        θ_size = 1e-3
        η_size = 50
        # γ = 0.5
        w, w_hist, u_hist = mvpo_egd(μ,Σ,γ[j],K,η_size)
        w = threshold(w,θ_size)
        # plt.plot(u_hist)
        # plt.grid()
        # plt.show()

        ### ASSIGN COVARIATES ###
        for i in range(N_IU):
            obs_X = np.array([log_mktcap[n,i],proof_of_work[n,i],beta_market[n,i],momentum_1w[n,i],θ[j],γ[j]])
            obs_Z = np.array([log_supply[n,i],proof_of_work[n,i],beta_market[n,i],momentum_1w[n,i],θ[j],γ[j]])
            y.append(w[i])
            X.append(obs_X)
            Z.append(obs_Z)
                
    print(n)
    
y = np.asarray(y)
X = np.asarray(X)
Z = np.asarray(Z)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


# Generalized Method of Moments

In [9]:
def GMM_obj(β,y,X,Z):
    n, d = np.shape(X)
    m = np.zeros(d)
    for i in range(n):
        m += (((y[i]/np.exp(β@X[i,:]))-1)*Z[i,:])/n
    return la.norm(m)**2

def GMM_obj_grad(β,y,X,Z,δ):
    n, d = np.shape(X)
    g = np.zeros(d)
    for i in range(d):
        β_R = β+δ*np.eye(N=1,M=d,k=i)[0]
        β_L = β-δ*np.eye(N=1,M=d,k=i)[0]
        g[i] = (GMM_obj(β_R,y,X,Z)-GMM_obj(β_L,y,X,Z))/(2*δ)
    return g

In [10]:
import scipy.optimize as opt

n, d = X.shape
β_init = np.zeros(d)

res = opt.minimize(GMM_obj,β_init,args=(y,X,Z))

In [11]:
res.fun

1.517476280725725e-06

In [12]:
res.x

array([-0.22671254, -1.60030764,  0.63007865,  0.0664751 , -0.27080513,
       -0.26538019])

In [13]:
β_star = res.x
G = np.zeros((d,d))
Ω = np.zeros((d,d))
for i in range(n):
    G += (-y[i]/np.exp(β_star@X[i,:]))*np.outer(Z[i,:],X[i,:])/n
    Ω += ((y[i]/np.exp(β_star@X[i,:]))-1)**2*np.outer(Z[i,:],Z[i,:])/n
GT = np.transpose(G)

Σ = la.inv(GT@G)@(GT@Ω@G)@la.inv(GT@G)/n

In [14]:
SE = np.sqrt(np.diag(Σ))
SE

array([0.00721927, 0.08145384, 0.12380028, 0.0042205 , 0.03711582,
       0.09124385])

In [15]:
t = β_star/np.sqrt(np.diag(Σ))
t

array([-31.40380435, -19.64680332,   5.08947695,  15.75053571,
        -7.29621768,  -2.90847215])